# ***Импорт библиотек***

In [ ]:
!pip3 install psycopg2-binary

In [ ]:
import os
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.feature_extraction.text import TfidfVectorizer #лемматизация и стемминг уже внутри вшиты
import pandas as pd
from sqlalchemy import create_engine
from catboost import CatBoostClassifier

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN


# Создаем датасет с лайками пользователей и вытаскиваем фитчи из взаимодействий

In [ ]:
# Создадим переменную connection_path для того чтобы подключаться к базе данных не указывая явно в коде логин и пароль от БД

config_file = "config.txt"
with open(config_file, "r") as f:
    config_data = f.readlines()

config = {}
for line in config_data:
    key, value = line.strip().split("=")
    config[key] = value
    
connection_path = f"postgresql://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

In [ ]:
def get_data_from_sql(query:str):
    conn_uri = connection_path
    df = pd.read_sql(query, conn_uri)
    return df

In [ ]:
def save_data_to_sql(df:DataFrame, table_name: str):
    
    engine = create_engine(
        connection_path
    )

    df.to_sql(table_name, con=engine, index=False, if_exists='replace')

In [ ]:
query_post_seen_and_liked_by_user = '''
    SELECT
        f.user_id,
        p.topic,
        COUNT(CASE WHEN f.action = 'view' THEN 1 ELSE NULL END) AS seen_posts,
        COUNT(CASE WHEN f.action = 'like' THEN 1 ELSE NULL END) AS liked_posts
    FROM
        public.feed_data AS f
    JOIN
        public.post_text_df AS p ON f.post_id = p.post_id
    GROUP BY
        f.user_id, p.topic
'''
df_most_liked_posts_by_topic = get_data_from_sql(query_post_seen_and_liked_by_user)

In [ ]:
# Группировка данных по 'user_id' и выбор топ-3 для каждой группы
top_3_per_user = df_most_liked_posts_by_topic.sort_values(by='liked_posts', ascending=False).groupby('user_id').head(3)
# Создание новой колонки 'top_3' с темами
df_most_liked_posts_by_topic['top_3'] = top_3_per_user.groupby('user_id')['topic'].agg(list).reindex(df_most_liked_posts_by_topic['user_id']).tolist()

In [ ]:
df_most_liked_posts_by_topic.head(8)

In [ ]:
query_likes_by_user = '''
    SELECT p.user_id, ARRAY_AGG(DISTINCT p.post_id) AS liked_posts
    FROM public.feed_data AS p
    WHERE p.target = 1
    GROUP BY p.user_id
'''
df_liked_posts = get_data_from_sql(query_likes_by_user)

In [ ]:
# df_most_liked_posts_by_topic нет дубликатов по 'user_id'
df_most_liked_posts_by_topic_ = df_most_liked_posts_by_topic.drop_duplicates(subset='user_id').copy()
# выполните мердж
df_liked_posts = pd.merge(df_liked_posts, df_most_liked_posts_by_topic_[['user_id', 'top_3']], on='user_id', how='left')

In [ ]:
df_liked_posts.head(2)

In [ ]:
save_data_to_sql(df_liked_posts,'koryakovda_features_users_actions')

# ***Загрузка и обработка фитчей и выгрузка для таблицы user***

In [ ]:
def users_average_age_per_city(df):
    av_age = df_user_data.groupby('city')['age'].mean()
    df['av_age_per_city'] = df['city'].map(av_age)
    return df

In [ ]:
def count_users_in_country(df):
    count = df['country'].value_counts()  # Count the number of users in each country
    df['users_in_country'] = df['country'].map(count)  # Map the counts back to the original DataFrame
    return df

In [ ]:
def user_age_split(df):
    bins = [0, 12, 17, 24, 34, 44, 54, 64, float('inf')]  # Определение границ бинов
    labels = ['Дети','Подростки', 'Молодежь', 'Молодые взрослые', 'Взрослые', 'Средний возраст', 'Старшее поколение', 'Пожилые']  # Названия групп
    df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)
    
    return df

In [ ]:
def user_feature_creation(df):
    df = user_age_split(df)
    df = count_users_in_country(df)
    df = users_average_age_per_city(df)
    
    return df

In [ ]:
df_user_data = get_data_from_sql("SELECT * FROM public.user_data")

In [ ]:
df_user_data = user_feature_creation(df_user_data)

In [ ]:
df_user_data.head(2)

In [ ]:
save_data_to_sql(df_user_data,'koryakovda_features_users')

## ***Загрузка и обработка фитчей и выгрузка для таблицы posts с использованием transformer (векторизуем тексты не через tfidf, а через distilbert)***

In [ ]:
df_post_text_for_nn = get_data_from_sql("SELECT * FROM public.post_text_df")

In [ ]:
df_post_text_for_nn

In [ ]:
### Сделаем эмбеддинги постов с помощью моделей трансформеров

from transformers import AutoTokenizer
from transformers import BertModel  # https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
from transformers import RobertaModel  # https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaModel
from transformers import DistilBertModel  # https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertModel


def get_model(model_name):
    assert model_name in ['bert', 'roberta', 'distilbert']

    checkpoint_names = {
        'bert': 'bert-base-cased',  # https://huggingface.co/bert-base-cased
        'roberta': 'roberta-base',  # https://huggingface.co/roberta-base
        'distilbert': 'distilbert-base-cased'  # https://huggingface.co/distilbert-base-cased
    }

    model_classes = {
        'bert': BertModel,
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])

In [ ]:
tokenizer, model = get_model('distilbert')

In [ ]:
model

In [ ]:
### Сделаем датасет для постов

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding


class PostDataset(Dataset):
    def __init__(self, texts, tokenizer):
        super().__init__()

        self.texts = tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            return_token_type_ids=False,
            return_tensors='pt',
            truncation=True,
            padding=True
        )
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        return {'input_ids': self.texts['input_ids'][idx], 'attention_mask': self.texts['attention_mask'][idx]}

    def __len__(self):
        return len(self.texts['input_ids'])
    
    
dataset = PostDataset(df_post_text_for_nn['text'].values.tolist(), tokenizer)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loader = DataLoader(dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=False)

In [ ]:
import torch
from tqdm import tqdm


@torch.inference_mode()
def get_embeddings_labels(model, loader):
    model.eval()
    
    total_embeddings = []
    
    for batch in tqdm(loader):
        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}

        embeddings = model(**batch)['last_hidden_state'][:, 0, :]

        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)
print(torch.cuda.get_device_name())

model = model.to(device)

In [ ]:
embeddings = get_embeddings_labels(model, loader).numpy()

embeddings

In [ ]:
### Пытаемся кластеризовать тексты как и делали до этого

centered = embeddings - embeddings.mean()

pca = PCA(n_components=50)
pca_decomp = pca.fit_transform(centered)

In [ ]:
df_post_text_for_nn.head(4)

In [ ]:
n_clusters = 16

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(pca_decomp)

df_post_text_for_nn['kmeans_labels'] = kmeans.labels_

dists_columns = [f'distance_to_cluster_{i}' for i in range(n_clusters)]

dists_df = pd.DataFrame(
    data=kmeans.transform(pca_decomp),
    columns=dists_columns
)

dists_df.head()

In [ ]:
df_post_text_for_nn = pd.concat((df_post_text_for_nn, dists_df), axis=1)

df_post_text_for_nn.head(2)

In [ ]:
save_data_to_sql(df_post_text_for_nn, 'koryakovda_features_post_transformer')

# ***Обучение модели***

In [ ]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(connection_path)
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)


def load_features() -> pd.DataFrame:
    query = 'SELECT * FROM public.feed_data LIMIT 2000000'
    return batch_load_sql(query)

In [ ]:
# Выгружаем табличку взаимодействий (размер корректируем для обучения)
user_post_iteractions = load_features()

In [ ]:
user_post_iteractions.head(3)

In [ ]:
# Выгружаем обработанные таблички из SQL
features_users_df = get_data_from_sql("SELECT * FROM koryakovda_features_users")
features_post_df_transformer = get_data_from_sql("SELECT * FROM koryakovda_features_post_transformer")
features_users_actions_df = get_data_from_sql("SELECT * FROM koryakovda_features_users_actions")

In [ ]:
# Обьединяем датасеты в один большой для исследования
def create_dataset(user_post_iteractions, features_users_df, features_post_df_transformer, features_users_actions_df):
    step_one = pd.merge(user_post_iteractions, features_users_df, on='user_id')
    step_two = pd.merge(step_one, features_post_df_transformer, on='post_id')
    step_three = pd.merge(step_two, features_users_actions_df[['user_id', 'top_3']], on='user_id')

    return step_three

In [ ]:
dataset = create_dataset(user_post_iteractions,features_users_df,features_post_df_transformer,features_users_actions_df)

***Добавление новых фитчей в итоговый датасет на основе временных данных***

In [ ]:
# Выделение часа и дня недели  - предположение что в зависимости от дня недели и часа дня утро\день\вечер - человека интересуюр разные потсы
def create_time_features(df):
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.day_name()
    df['month'] = df['timestamp'].dt.month
    return df

In [ ]:
dataset = create_time_features(dataset)

***Построим графики нашего предположения - изменение отношения показов поста к количеству лайков от дня недели и часа дня***

In [ ]:
def plot_show_like_ratio_by_day(df, days_order=None):
    # Группировка данных по теме, дню недели и подсчет количества показанных и лайкнутых постов
    grouped_data = df.groupby(['topic', 'day_of_week', 'action']).size().reset_index(name='count')

    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    # Указание порядка дней недели
    grouped_data['day_of_week'] = pd.Categorical(grouped_data['day_of_week'], categories=days_order, ordered=True)

    # Создание сводной таблицы с количеством показанных и лайкнутых постов по темам и дням недели
    pivot_table = pd.pivot_table(grouped_data, values='count', index=['topic', 'day_of_week'], columns='action', fill_value=0)

    # Вычисление отношения количества показанных постов к количеству лайкнутых постов
    pivot_table['view_like_ratio'] = pivot_table['view'] / pivot_table['like']

    # Построение графика
    plt.figure(figsize=(12, 6))
    sns.lineplot(x='day_of_week', y='view_like_ratio', hue='topic', data=pivot_table.reset_index(), marker='o')
    plt.title('Show-to-Like Ratio by Topic and Day of Week')
    plt.xlabel('Day of Week')
    plt.ylabel('Show-to-Like Ratio')
    plt.legend(title='Topic', loc='upper right')
    plt.xticks(rotation=45, ha='right')
    plt.show()

# Пример использования функции с вашим датасетом

plot_show_like_ratio_by_day(dataset, days_order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])


In [ ]:
def plot_show_like_ratio_by_hour(df):
    # Группировка данных по теме, часу дня и подсчет количества показанных и лайкнутых постов
    grouped_data = df.groupby(['topic', 'hour', 'action']).size().reset_index(name='count')

    # Создание сводной таблицы с количеством показанных и лайкнутых постов по темам и часам дня
    pivot_table = pd.pivot_table(grouped_data, values='count', index=['topic', 'hour'], columns='action', fill_value=0)

    # Вычисление отношения количества показанных постов к количеству лайкнутых постов
    pivot_table['view_like_ratio'] = pivot_table['view'] / pivot_table['like']

    # Построение графика
    plt.figure(figsize=(12, 6))
    sns.lineplot(x='hour', y='view_like_ratio', hue='topic', data=pivot_table.reset_index(), marker='o')
    plt.title('Show-to-Like Ratio by Topic and Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Show-to-Like Ratio')
    plt.legend(title='Topic', loc='upper right')
    plt.xticks(grouped_data['hour'].unique())  
    plt.show()

plot_show_like_ratio_by_hour(dataset)


In [ ]:
pd.set_option('display.max_columns', None)
dataset.head(2)
# pd.reset_option('display.max_columns')

***Подготовка к построению модели***

In [ ]:
# Модель будем строить по столбцу target поэтому сразу удаляем из датасета взаимодействия-повторы где like target = 0 (там view = 1)
dataset = dataset[dataset['action'] != 'like']
dataset.head(2)

In [ ]:
# Сортируем датасет для дальнейшего деления на трейн и тест по timestamp
dataset = dataset.sort_values(by = 'timestamp')
dataset.head(2)

In [ ]:
# Выделяем категориальный фитчи
cat_features = ['gender','country','city','exp_group','os','source','age_group','topic','kmeans_labels','day_of_week','hour', 'month', 'top_3']

In [ ]:
# Делим на трейн и тест выделив для трейна 80% и теста 20% выборки
num_rows = int(len(dataset) * 0.2)
train_set = dataset.iloc[:-num_rows]
test_set = dataset.iloc[-num_rows:]

In [ ]:
# Создадим список для каждого юзера с постами которые были лайкнуты на трейне чтобы удалить их на тесте чтобы модель их не рекомендовала. 
# т.к. рекоменовать человеку снова что-то уже лайкнутое в трейне как будто не логично (подглядываем в ответ)
liked_posts_by_users_train = train_set[train_set['target'] == 1][['user_id', 'post_id']].drop_duplicates()

In [ ]:
# Уберем из теста лайкнутые в трейне посты
merged_data = test_set.merge(liked_posts_by_users_train, on=['user_id', 'post_id'], how='left', indicator=True)
filtered_test_set = merged_data[merged_data['_merge'] == 'left_only']
filtered_test_set = filtered_test_set.drop(columns=['_merge'])

In [ ]:
filtered_test_set.head(2)

***Работа по предсказанию***

In [ ]:
# поделим данные на фитчи и таргет
X_train = train_set.drop('target', axis=1)
X_test = filtered_test_set.drop('target', axis=1)

y_train = train_set['target']
y_test = filtered_test_set['target']

In [ ]:
# Удаляем ненужные стлобцы, создаем новый датасет для работы
X_train_cut = X_train.drop(['timestamp','text','user_id','post_id','action'],axis =1)
X_test_cut = X_test.drop(['timestamp','text','user_id','post_id','action'],axis =1)

In [ ]:
X_train_cut.head(2)

In [ ]:
# Обучаем Catboost
catboost = CatBoostClassifier()
catboost.fit(X_train_cut,y_train, cat_features=cat_features)

In [ ]:
#Считаем предсказание
X_test_cut['content_prediction'] = catboost.predict_proba(X_test_cut)[:, 1]

# Добавляем истиный target, user_id, post_id для дальнейшего подсчета метрики
X_test_cut['target'] = y_test
X_test_cut['user_id'] = X_test['user_id']
X_test_cut['post_id'] = X_test['post_id']

X_test_cut.head(2)

***Оцениваем наш HitRate@5*** \
Имеет ли смысл оценка на тесте если в тестовой выборке у нас не все посты для каждого пользователя а только малая часть?  
Как будто нет, но все равно сделаем для консистенции, реально тестить модель будем уже в сервисе

In [ ]:
def hitrate_at_5(X_test_cut, limit):
    hits = []
    for user in X_test_cut['user_id'].unique():
        part = X_test_cut[X_test_cut['user_id']== user]
        part = part.sort_values('content_prediction',ascending=False)[:limit] # выбираем топ N рекомендаций
        if part['target'].sum() >= 1:
            hit = 1
        else:
            hit = 0
        hits.append(hit)
    hitrate_at_5 = sum(hits) / len(X_test_cut['user_id'].unique())
    
    return hitrate_at_5

In [ ]:
# Оцениваем качество модели
hitrate_at_5(X_test_cut,5)

***Сохранение / загрузка модели***

In [ ]:
catboost.save_model('model_nn', format="cbm")

In [ ]:
# def get_model_path(path: str) -> str:
#     if os.environ.get("IS_LMS") == "1":  # проверяем где выполняется код в лмс, или локально.
#         MODEL_PATH = '/workdir/user_input/model'
#     else:
#         MODEL_PATH = path
#     return MODEL_PATH

# def load_models():
#     model_path = get_model_path("./catboost_model")  # Предпложим что данные в той же директории
#     from_file = CatBoostClassifier()
#     model = from_file.load_model(model_path)
#     return model

In [ ]:
def get_model_path(model_version: str) -> str:
    """
    Здесь мы модицифируем функцию так, чтобы иметь возможность загружать
    обе модели. При этом мы могли бы загружать и приципиально разные
    модели, так как никак не ограничены тем, какой код использовать.
    """
    print(os.environ)
    if (
        os.environ.get("IS_LMS") == "1"
    ):  # проверяем где выполняется код в лмс, или локально. Немного магии
        model_path = f"/workdir/user_input/{model_version}"
    else:
        model_path = (f"./{model_version}")
    return model_path


def load_models(model_version: str):
    model_path = get_model_path(model_version)
    loaded_model = CatBoostClassifier()
    loaded_model.load_model(model_path)
    return loaded_model

# ***Проверка работы на примере конкретного юзера. Формирование сервиса - полного цикла предсказаний***

Берем любого юзера с фитчами прикручиваем к каждому посту и убираем лайкнутые посты чтобы их не рекомендовать 

In [ ]:
# Выгружаем обработанные таблички из SQL
features_users_df = get_data_from_sql("SELECT * FROM koryakovda_features_users")
features_post_df_transformer = get_data_from_sql("SELECT * FROM koryakovda_features_post_transformer")
features_users_actions_df = get_data_from_sql("SELECT * FROM koryakovda_features_users_actions")

In [ ]:
features_users_df.head(2)

In [ ]:
features_post_df_transformer.head(2)

In [ ]:
features_users_actions_df.head(2)

In [ ]:
# Для примера возьмем user_id = 202
user_features = features_users_df[features_users_df['user_id'] == 202].copy()
user_features

In [ ]:
# Обрабатываем список всех постов которые лайкнул пользователь, чтобы не рекоендовать их потом
user_likes = features_users_actions_df[features_users_actions_df['user_id']==202]['liked_posts'].iloc[0]
user_likes

In [ ]:
user_likes = user_likes.strip('{}').split(',')

In [ ]:
user_likes = [int(x) for x in user_likes]
print(user_likes)

In [ ]:
# Удаляем из датасета со всеми обработанными постами посты уже лайкнутые пользователем чтобы не рекомендовать их снова
posts_features = features_post_df_transformer[~features_post_df_transformer['post_id'].isin(user_likes)].copy()
posts_features.head(2)

In [ ]:
# Задаем время рекомендации как входной параметр
time = datetime(year=2021, month=1, day=3, hour=14)

In [ ]:
# добавляем временные фитчи в финальный датасет
posts_features['hour'] = time.hour
posts_features['day_of_week'] = time.strftime("%A")
posts_features['month'] = time.month

In [ ]:
# мерджим выбранного в запросе юзера к каждому посту
final_df = posts_features.assign(**user_features.iloc[0])

In [ ]:
# Присоединим top3 категории к датасету
final_df = pd.merge(final_df,features_users_actions_df[['user_id','top_3']], on = 'user_id', how = 'left')

In [ ]:
final_df.head(10)

In [ ]:
# удаляем из финального датасета ненужные столбцы
final_df = final_df.drop(['text', 'user_id'], axis=1)

In [ ]:
# задаем post_id как индекс нашего финального датасета
final_df.set_index('post_id',inplace = True)

In [ ]:
final_df

In [ ]:
# Приводим трейн датасет и рабочий в соответствие по порядку строк
desired_column_order = X_train_cut.columns.tolist()
final_df = final_df[desired_column_order]
print(desired_column_order)

In [ ]:
# Загрузим обученную ранее модель с использованием функции из степпа 2
model = load_models('model_nn')

In [ ]:
# считаем предсказания
preds = model.predict_proba(final_df)[:, 1]
preds

In [ ]:
# выбираем топ 5 постов по предсказаниям вероятности лайка
preds = pd.DataFrame(model.predict_proba(final_df)[:, 1], columns=['probability'], index=final_df.index)
top5_predictions = preds.nlargest(5, 'probability').index
top5_predictions

In [ ]:
# загрузим класс для типизации выходного ответа
from pydantic import BaseModel

class PostGet(BaseModel):
    id: int
    text: str
    topic: str

    class Config:
        orm_mode = True

In [ ]:
# Выдаем 5 постов рекомендаций по юзеру
recommended_posts = []
for post_id in top5_predictions:
    post_data = features_post_df_transformer.loc[features_post_df_transformer['post_id'] == post_id]
    if not post_data.empty:
        post = PostGet(
            id=post_id,
            text=post_data['text'].iloc[0],
            topic=post_data['topic'].iloc[0]
        )
        recommended_posts.append(post)

recommended_posts


# ***Итоговый сервис:***

In [ ]:
# Так должн выглядеть мой эндпоинт для предсказания 5 постов по пользователю
import os
from typing import List
from fastapi import FastAPI
from datetime import datetime

app = FastAPI()

# ***Загрузка модели CatBoostClassifier***
model = load_models('model_nn')
# передадим порядок колонок из трейна, чтобы далее датасет для предсказаний сделать таким же
desired_column_order = ['gender','age','country','city','exp_group','os','source','age_group','users_in_country','av_age_per_city',
                        'topic','word_count','tfidf_sum','tfidf_mean','tfidf_max','kmeans_labels','distance_to_cluster_0',
                        'distance_to_cluster_1','distance_to_cluster_2','distance_to_cluster_3','distance_to_cluster_4','distance_to_cluster_5',
                        'distance_to_cluster_6','distance_to_cluster_7','distance_to_cluster_8','distance_to_cluster_9','distance_to_cluster_10',
                        'distance_to_cluster_11','distance_to_cluster_12','distance_to_cluster_13','distance_to_cluster_14','distance_to_cluster_15',
                        'top_3','hour','day_of_week']

# ***Загрузка обработанных фитчей для таблицы user***
# df_user_data = get_data_from_sql("SELECT * FROM koryakovda_features_users")
features_users_df = get_data_from_sql("SELECT * FROM koryakovda_features_users")

# ***Загрузка обработанных фитчей для таблицы posts***
# df_posts = get_data_from_sql("SELECT * FROM koryakovda_features_post")
features_post_df_transformer = get_data_from_sql("SELECT * FROM koryakovda_features_post_transformer")

# ***Загрузка датасет с лайками пользователей и фитчами из взаимодействий***
# df_liked_posts = get_data_from_sql("SELECT * FROM koryakovda_features_users_actions")
features_users_actions_df = get_data_from_sql("SELECT * FROM koryakovda_features_users_actions")



@app.get("/post/recommendations/", response_model=List[PostGet])
def get_post_recommendations(id: int, time: datetime, limit: int = 5):
    

    user_features = features_users_df[features_users_df['user_id'] == id].copy()
    user_likes = features_users_actions_df[features_users_actions_df['user_id'] == id]['liked_posts'].iloc[0]
    user_likes = user_likes.strip('{}').split(',')
    user_likes = [int(x) for x in user_likes]
    posts_features = features_post_df_transformer[~features_post_df_transformer['post_id'].isin(user_likes)].copy()
    posts_features['hour'] = time.hour
    posts_features['day_of_week'] = time.strftime("%A")
    posts_features['month'] = time.month
    final_df = posts_features.assign(**user_features.iloc[0])
    final_df = pd.merge(final_df, features_users_actions_df[['user_id', 'top_3']], on='user_id', how='left')
    final_df = final_df.drop(['text', 'user_id'], axis=1)
    final_df.set_index('post_id', inplace=True)
    final_df = final_df[desired_column_order]
    preds = pd.DataFrame(model.predict_proba(final_df)[:, 1], columns=['probability'], index=final_df.index)
    top5_predictions = preds.nlargest(5, 'probability').index
    
    recommended_posts = []
    for post_id in top5_predictions:
        post_data = features_post_df_transformer.loc[features_post_df_transformer['post_id'] == post_id]
        if not post_data.empty:
            post = PostGet(
                id=post_id,
                text=post_data['text'].iloc[0],
                topic=post_data['topic'].iloc[0]
            )
            recommended_posts.append(post)

    return recommended_posts

## Весь сервис app.py для того чтобы протестить обе модели

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

SQLALCHEMY_DATABASE_URL = connection_path

engine = create_engine(SQLALCHEMY_DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()


In [ ]:
import datetime

from pydantic import BaseModel


class UserGet(BaseModel):
    id: int
    gender: int
    age: int
    country: str
    city: str
    exp_group: int
    os: str
    source: str

    class Config:
        orm_mode = True


class PostGet(BaseModel):
    id: int
    text: str
    topic: str

    class Config:
        orm_mode = True

class Response(BaseModel):
    exp_group: str
    recommendations: List[PostGet]

class FeedGet(BaseModel):
    user_id: int
    post_id: int
    action: str
    time: datetime.datetime
    user: UserGet
    post: PostGet

    class Config:
        orm_mode = True


In [ ]:
from fastapi import FastAPI, HTTPException, Depends
from sqlalchemy.orm import Session
from sqlalchemy import desc, func, create_engine
from typing import List
import os
from datetime import datetime
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
import hashlib


app = FastAPI()


def get_db():
    with SessionLocal() as db:
        return db

def get_model_path(model_version: str) -> str:
    """
    Здесь мы модицифируем функцию так, чтобы иметь возможность загружать
    обе модели. При этом мы могли бы загружать и приципиально разные
    модели, так как никак не ограничены тем, какой код использовать.
    """
    print(os.environ)
    if (
        os.environ.get("IS_LMS") == "1"
    ):  # проверяем где выполняется код в лмс, или локально. Немного магии
        model_path = f"/workdir/user_input/{model_version}"
    else:
        # Пробуем загрузить модель из текущей директории (если такой модели не будет будет просто присвоено None)
        model_path = f"./{model_version}"
    return model_path

def load_models(model_version: str):
    model_path = get_model_path(model_version)
    loaded_model = CatBoostClassifier()
    loaded_model.load_model(model_path)
    return loaded_model

# ***Загрузка и обработка фитчей и выгрузка для таблицы user***
def get_data_from_sql(query: str):
    conn_uri = connection_path
    df = pd.read_sql(query, conn_uri)
    return df

SALT = "my_salt"

def get_exp_group(user_id: int) -> str:
    value_str = str(id) + SALT
    value_num = int(hashlib.md5(value_str.encode()).hexdigest(), 16)
    percent = value_num % 100
    if percent < 50:
        return "control"
    elif percent < 100:
        return "test"
    return "unknown"

# ***Загрузка моделей CatBoostClassifier***
model_control = load_models('model_tfidf')
model_test = load_models('model_nn')

# передадим порядок колонок из трейна, чтобы далее датасет для предсказаний сделать таким же
desired_column_order = ['gender','age','country','city','exp_group','os','source','age_group','users_in_country','av_age_per_city',
                        'topic','word_count','tfidf_sum','tfidf_mean','tfidf_max','kmeans_labels','distance_to_cluster_0',
                        'distance_to_cluster_1','distance_to_cluster_2','distance_to_cluster_3','distance_to_cluster_4','distance_to_cluster_5',
                        'distance_to_cluster_6','distance_to_cluster_7','distance_to_cluster_8','distance_to_cluster_9','distance_to_cluster_10',
                        'distance_to_cluster_11','distance_to_cluster_12','distance_to_cluster_13','distance_to_cluster_14','distance_to_cluster_15',
                        'top_3','hour','day_of_week']

# ***Загрузка обработанных фитчей для таблицы user***
# df_user_data = get_data_from_sql("SELECT * FROM koryakovda_features_users")
features_users_df = get_data_from_sql("SELECT * FROM koryakovda_features_users")

# ***Загрузка обработанных фитчей для таблицы posts***
# df_posts = get_data_from_sql("SELECT * FROM koryakovda_features_post")
features_post_df_transformer = get_data_from_sql("SELECT * FROM koryakovda_features_post_transformer")

# ***Загрузка обработанных фитчей для таблицы posts***
# df_posts = get_data_from_sql("SELECT * FROM koryakovda_features_post")
features_post_df = get_data_from_sql("SELECT * FROM koryakovda_features_post")

# ***Загрузка датасет с лайками пользователей и фитчами из взаимодействий***
# df_liked_posts = get_data_from_sql("SELECT * FROM koryakovda_features_users_actions")
features_users_actions_df = get_data_from_sql("SELECT * FROM koryakovda_features_users_actions")


# ***FastAPI***
@app.get("/user/{id}", response_model=UserGet)
def get_user_id(id: int, db: Session = Depends(get_db)):
    result = db.query(User).filter(User.id == id).first()
    if result is None:
        raise HTTPException(404, 'id not found')
    else:
        return result


@app.get("/post/{id}", response_model=PostGet)
def get_post_id(id: int, db: Session = Depends(get_db)):
    result = db.query(Post).filter(Post.id == id).first()
    if result is None:
        raise HTTPException(404, 'id not found')
    else:
        return result


@app.get("/user/{id}/feed", response_model=List[FeedGet])
def user_feed(id: int, limit: int = 10, db: Session = Depends(get_db)):
    result = db.query(Feed).filter(Feed.user_id == id).order_by(desc(Feed.time)).limit(limit).all()
    return result


@app.get("/post/{id}/feed", response_model=List[FeedGet])
def post_feed(id: int, limit: int = 10, db: Session = Depends(get_db)):
    result = db.query(Feed).filter(Feed.post_id == id).order_by(desc(Feed.time)).limit(limit).all()
    return result

@app.get("/post/recommendations/", response_model=Response)
def get_post_recommendations(id: int, time: datetime, limit: int = 5):
    # Выбираем группу пользователи
    user_group = get_exp_group(id=id)

    # Выбираем нужную модель
    if user_group == "control":
        model = model_control
        posts_features = features_post_df[~features_post_df['post_id'].isin(user_likes)].copy()
    elif user_group == "test":
        model = model_test
        posts_features = features_post_df_transformer[~features_post_df_transformer['post_id'].isin(user_likes)].copy()
    else:
        raise ValueError("unknown group")

    user_features = features_users_df[features_users_df['user_id'] == id].copy()
    user_likes = features_users_actions_df[features_users_actions_df['user_id'] == id]['liked_posts'].iloc[0]
    user_likes = user_likes.strip('{}').split(',')
    user_likes = [int(x) for x in user_likes]
    posts_features['hour'] = time.hour
    posts_features['day_of_week'] = time.strftime("%A")
    posts_features['month'] = time.month
    final_df = posts_features.assign(**user_features.iloc[0])
    final_df = pd.merge(final_df, features_users_actions_df[['user_id', 'top_3']], on='user_id', how='left')
    final_df = final_df.drop(['text', 'user_id'], axis=1)
    final_df.set_index('post_id', inplace=True)
    final_df = final_df[desired_column_order]
    preds = pd.DataFrame(model.predict_proba(final_df)[:, 1], columns=['probability'], index=final_df.index)
    top5_predictions = preds.nlargest(5, 'probability').index

    recommended_posts = []
    for post_id in top5_predictions:
        if user_group == "control":
            post_data = features_post_df.loc[features_post_df['post_id'] == post_id]
        elif user_group == "test":
            post_data = features_post_df_transformer.loc[features_post_df_transformer['post_id'] == post_id]

        if not post_data.empty:
            post = PostGet(
                id=post_id,
                text=post_data['text'].iloc[0],
                topic=post_data['topic'].iloc[0]
            )
            recommended_posts.append(post)

    return Response(recommendations=recommended_posts, exp_group=user_group)